In [1]:
import sys
import os

# Add parent directory and sibling directories to Python path
sys.path.append(os.path.abspath('..'))  # Go up to parent directory
  # Add mlp directory
  # Add tensort directory

# Import TensorT class
from TensorT.tensor_scratch import TensorT

# Import MLP and related classes
from mlp_latest.mlp import MLP
from mlp_latest.ActivationFunctions import ActivationFunction
from mlp_latest.LossFunctions import LossFunction
from mlp_latest.WeightInitialization import WeightInitializer

# Import additional libraries for testing
import numpy as np
from sklearn.datasets import make_moons, make_classification
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import time


In [2]:
# Test 1: Basic TensorT functionality
print("=== Testing TensorT ===")
t1 = TensorT([[1.0, 2.0], [3.0, 4.0]])
t2 = TensorT([[0.5, 1.0], [1.5, 2.0]])
t3 = t1 + t2
print(f"Addition result: {t3.data}")

# Test 2: Initialize MLP
print("\n=== Testing MLP Initialization ===")
mlp = MLP(
    input_size=2,
    hidden_layers=[4, 3],
    output_size=1,
    weight_initialization='he_normal',
    activation_func='relu',
    loss_function='binary_cross_entropy_loss',
    learning_rate=0.01
)

print("MLP initialized successfully!")
print(f"Weight types: {[type(w).__name__ for w in mlp.weights]}")
print(f"Bias types: {[type(b).__name__ for b in mlp.biases]}")

# Test 3: Forward pass
print("\n=== Testing Forward Pass ===")
X = TensorT([[1.0, 2.0], [0.5, 1.5]])  # 2x2 input
output = mlp.forward(X)
print(f"Output shape: {output.shape}")
print(f"Output type: {type(output).__name__}")

# Test 4: Complete training workflow
print("\n=== Testing Complete Training ===")
# Generate simple dataset
X_data, y_data = make_moons(n_samples=100, noise=0.1, random_state=42)
X_train = TensorT(X_data.T.tolist())  # Convert to TensorT format (2, 100)
y_train = TensorT([y_data.tolist()])   # Convert to TensorT format (1, 100)

print(f"Training data - X: {X_train.shape}, y: {y_train.shape}")

# Train for a few epochs
costs = mlp.train(X_train, y_train, epochs=10, print_cost_every=2)
print(f"Training completed! Final cost: {costs[-1]:.6f}")

# Test predictions
predictions = mlp.predict(X_train)
print(f"Predictions shape: {predictions.shape}")
print(f"First 5 predictions: {predictions.data[0][:5]}")


=== Testing TensorT ===
Addition result: [[1.5, 3.0], [4.5, 6.0]]

=== Testing MLP Initialization ===
MLP initialized successfully.
  - Architecture: [2, 4, 3, 1]
  - Hidden Activation: relu
  - Output Activation (from loss): sigmoid
  - Weight Initialization: he_normal
  - Loss Function: binary_cross_entropy_loss
MLP initialized successfully!
Weight types: ['TensorT', 'TensorT', 'TensorT']
Bias types: ['TensorT', 'TensorT', 'TensorT']

=== Testing Forward Pass ===
Initial A type: <class 'TensorT.tensor_scratch.TensorT'>
Initial A class: <class 'TensorT.tensor_scratch.TensorT'>

Layer 1:
  W type: <class 'TensorT.tensor_scratch.TensorT'>
  A type: <class 'TensorT.tensor_scratch.TensorT'>
  isinstance(W, TensorT): True
  isinstance(A, TensorT): True
  After activation A type: <class 'TensorT.tensor_scratch.TensorT'>

Layer 2:
  W type: <class 'TensorT.tensor_scratch.TensorT'>
  A type: <class 'TensorT.tensor_scratch.TensorT'>
  isinstance(W, TensorT): True
  isinstance(A, TensorT): True

In [4]:
"""
Comprehensive test suite for TensorT + MLP
Covers forward, activations, loss, gradients, training
"""

import math
import numpy as np

from sklearn.datasets import make_moons

# -------------------------------
# Utility for numerical gradient check
# -------------------------------
def numerical_gradient_check(mlp, X, Y, epsilon=1e-5):
    """
    Compare autograd gradient vs finite difference gradient for one parameter.
    """
    W = mlp.weights[0]  # check first weight matrix
    grads_autograd = None

    # Run forward + backward to get autograd grads
    AL = mlp.forward(X)
    loss = mlp.cost(Y, AL)
    loss.zero_grad()
    loss.backward()
    grads_autograd = np.array(W.grad)  # TensorT grad is list

    # Numerical gradient
    grads_numeric = np.zeros_like(grads_autograd)
    for i in range(W.shape[0]):
        for j in range(W.shape[1]):
            original_val = W.data[i][j]

            # w+ε
            W.data[i][j] = original_val + epsilon
            AL_pos = mlp.forward(X)
            L_pos = mlp.cost(Y, AL_pos).tsum()

            # w-ε
            W.data[i][j] = original_val - epsilon
            AL_neg = mlp.forward(X)
            L_neg = mlp.cost(Y, AL_neg).tsum()

            grads_numeric[i][j] = (L_pos - L_neg) / (2 * epsilon)

            # restore
            W.data[i][j] = original_val

    # Compare
    diff = np.linalg.norm(grads_autograd - grads_numeric) / (
        np.linalg.norm(grads_autograd + grads_numeric) + 1e-8
    )
    return diff, grads_autograd, grads_numeric


# -------------------------------
# Test 1: Basic TensorT operations
# -------------------------------
print("=== Test 1: TensorT ops ===")
a = TensorT([[1.0, 2.0], [3.0, 4.0]])
b = TensorT([[5.0, 6.0], [7.0, 8.0]])
print("a+b =", (a+b).data)
print("a*b =", (a*b).data)
print("a-b =", (a-b).data)
print("a/b =", (a/b).data)

# -------------------------------
# Test 2: Activations
# -------------------------------
print("\n=== Test 2: Activations ===")
print("Sigmoid(0) =", 1 / (1 + math.exp(0)))
print("ReLU(-1) =", max(0, -1), "ReLU(2) =", max(0, 2))
print("Tanh(0) =", math.tanh(0))

# -------------------------------
# Test 3: Loss function correctness
# -------------------------------
print("\n=== Test 3: Loss functions ===")
y_true = TensorT([[1.0, 0.0]])
y_pred = TensorT([[0.8, 0.2]])
# Manual BCE
bce_manual = -(math.log(0.8) + math.log(1 - 0.2)) / 2
cost, _ = LossFunction.get("binary_cross_entropy_loss")
bce_tensor = cost(y_true, y_pred).tmean()
print("Manual BCE:", bce_manual, " | Tensor BCE:", bce_tensor)

# -------------------------------
# Test 4: Forward pass small MLP
# -------------------------------
print("\n=== Test 4: Forward pass ===")
mlp = MLP(input_size=2, hidden_layers=[2], output_size=1,
          activation_func="relu", loss_function="binary_cross_entropy_loss",
          weight_initialization="he_normal", learning_rate=0.01)
X = TensorT([[1.0], [2.0]])
out = mlp.forward(X)
print("Forward output:", out)

# -------------------------------
# Test 5: Backprop gradient check
# -------------------------------
print("\n=== Test 5: Gradient check ===")
X = TensorT([[0.5], [1.5]])
Y = TensorT([[1.0]])
diff, grads_auto, grads_num = numerical_gradient_check(mlp, X, Y)
print("Gradient diff (should be ~1e-7):", diff)

# -------------------------------
# Test 6: Training on make_moons
# -------------------------------
print("\n=== Test 6: Training ===")
X_data, y_data = make_moons(n_samples=200, noise=0.2, random_state=0)
X_train = TensorT(X_data.T.tolist())
y_train = TensorT([y_data.tolist()])

mlp = MLP(input_size=2, hidden_layers=[4, 3], output_size=1,
          activation_func="relu", loss_function="binary_cross_entropy_loss",
          weight_initialization="he_normal", learning_rate=0.01)

costs = mlp.train(X_train, y_train, epochs=50, print_cost_every=10)

preds = mlp.predict(X_train).data[0]
acc = sum(int(p == y) for p, y in zip(preds, y_data)) / len(y_data)
print("Final training accuracy:", acc)


=== Test 1: TensorT ops ===
a+b = [[6.0, 8.0], [10.0, 12.0]]
a*b = [[5.0, 12.0], [21.0, 32.0]]
a-b = [[-4.0, -4.0], [-4.0, -4.0]]
a/b = [[0.2, 0.3333333333333333], [0.42857142857142855, 0.5]]

=== Test 2: Activations ===
Sigmoid(0) = 0.5
ReLU(-1) = 0 ReLU(2) = 2
Tanh(0) = 0.0

=== Test 3: Loss functions ===
Manual BCE: 0.2231435513142097  | Tensor BCE: 0.2231435513142097

=== Test 4: Forward pass ===
MLP initialized successfully.
  - Architecture: [2, 2, 1]
  - Hidden Activation: relu
  - Output Activation (from loss): sigmoid
  - Weight Initialization: he_normal
  - Loss Function: binary_cross_entropy_loss
Initial A type: <class 'TensorT.tensor_scratch.TensorT'>
Initial A class: <class 'TensorT.tensor_scratch.TensorT'>

Layer 1:
  W type: <class 'TensorT.tensor_scratch.TensorT'>
  A type: <class 'TensorT.tensor_scratch.TensorT'>
  isinstance(W, TensorT): True
  isinstance(A, TensorT): True
  After activation A type: <class 'TensorT.tensor_scratch.TensorT'>

Layer 2:
  W type: <class '